# Objectives
YWBAT
- perform a gridsearch in Keras

In [3]:
import warnings
import pandas as pd
import numpy as np
import string
import nltk

from nltk.corpus import stopwords
from nltk import word_tokenize, FreqDist
from sklearn.decomposition import PCA
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.datasets import fetch_20newsgroups


import keras
from keras import optimizers
from keras.layers import Dense

from gensim.models import Word2Vec


import matplotlib.pyplot as plt
import seaborn as sns

warnings.filterwarnings("ignore")

In [18]:
df_train = pd.read_csv("data/nlp_data_train.csv")
df_test = pd.read_csv("data/nlp_data_test.csv")
df_train.dropna(inplace=True)
df_test.dropna(inplace=True)

In [19]:
df_train.head()


,article,label,processed_article,cleaned_article_string
0,Note: These trial updates are summarized from ...,4,"['note', 'trial', 'updates', 'summarized', 're...",note trial updates summarized reports idaho st...
2,I'm trying to create a button that has both a ...,1,"['trying', 'create', 'button', 'label', 'bitma...",trying create button label bitmap using menubu...
3,"Hi. I've just finished reading S414, and have ...",4,"['hi', ""'ve"", 'finished', 'reading', 's414', '...",hi 've finished reading s414 several questions...
4,\nUnfortunately your phrasing is ambiguous. R...,0,"['unfortunately', 'phrasing', 'ambiguous', 're...",unfortunately phrasing ambiguous re-writing ca...
5,\n\n\tSeAL Team six should have handled it? D...,4,"['seal', 'team', 'six', 'handled', 'delta', 'f...",seal team six handled delta force batf enough ...


In [20]:
# Let's build a tfidf model

In [21]:
tfidf = TfidfVectorizer(max_features=10000)

In [22]:
tfidf_train = tfidf.fit_transform(df_train["cleaned_article_string"], y=df_train['label'])

In [23]:
tfidf_test = tfidf.transform(df_test['cleaned_article_string'])

In [24]:
tfidf_train.shape

(2756, 10000)